In [ ]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

## Built-in Tool - DuckDuckGo Search

In [ ]:
!pip install -U ddgs

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('Top news in Bangladesh Today')

print(results)

Most Popular English News Paper in Bangladesh . Get Latest English Breaking News , Sports, Crime, Entertainment, Business, Politics, Education, Opinion, Lifestyle, Photo Galleries, Science & Technology. Top 20 defaulters of 6 banks Debt recovery will take 333 years. Discover the updated list of popular Bangla newspapers in 2025, including top online and print media like Prothom Alo, Kaler Kantho, and Jugantor. Visit allbanglanewspaperbd.com for verified links and full details. Stay on top of Bangladesh latest developments on the ground with Al Jazeera’s fact-based news , exclusive video footage, photos and updated maps. All the latest content about Bangladesh from the BBC. Bangladesh Newspapers : All Bangla Newspapers (বাংলা সংবাদপত্র) & BD News Sites.Most national daily newspapers in Bangladesh are based in Dhaka, but major cities like Chittagong, Sylhet, Rajshahi, Rangpur, Khulna, Mymensingh, and Cumilla also have a few daily publications.


In [5]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Built-in Tool - Shell Tool

In [7]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

In [8]:
results = shell_tool.invoke('ls')

print(results)

Executing command:
 ls
sample_data



/usr/local/lib/python3.12/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


In [9]:
results = shell_tool.invoke('whoami')

print(results)

Executing command:
 whoami
root



/usr/local/lib/python3.12/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


## Custom Tools

# Method 1 - Using @tool

In [10]:
from langchain_core.tools import tool

In [11]:
# Step 1 - create a function

def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [12]:
# Step 2 - add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [13]:
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [14]:
result = multiply.invoke({"a":3, "b":5})

In [15]:
print(result)

15


In [16]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [17]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Method 2 - Using StructuredTool

In [18]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [19]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [20]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [21]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [22]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Method 3 - Using BaseTool Class

In [23]:
from langchain.tools import BaseTool
from typing import Type

In [24]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [25]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [26]:
multiply_tool = MultiplyTool()

In [27]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Toolkit

In [28]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [29]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [30]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
